In [103]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**INTRODUCTION**

**WAHT IS CLUSTERING?**

* grouping of data based on similarities in a data set

**What is unsupervised learning**

* type of algorithm that learn patetrn from unlabeled data
we used KMeans algorithm here.

**application of clustering**
* customer segmentation
* market segmentation
* health and image processing
* cumputer vision



**Brief intro of KMeans**

1. select how many clusters will be used as a parameter form the user
2. k center points are randomly selected
3. each data point is assigned to the related cluster according to its nearest center point
4. the center points are shifted by calculating new center point for each cluster
5. after the new center points are determined, step 3 and 4 are repeated until the system stabilizes

**WCSS AND Elbow method**
* wcss (within cluster sum of squares) is the sum of squares of the distances of each data point in all clusters to their respective centroids

* the Elbow method is for finding optimal k value

**EXPLORATORY DATA ANALYSIS**

In [104]:
# import packages

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [105]:
data_path = '/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv'

In [106]:
df = pd.read_csv(data_path)

In [107]:
df.head()

In [108]:
# ID is a unique number for each individuals, therefore we need to drop it
df.drop('CustomerID', axis = 1, inplace = True)
df.head()

In [109]:
df.shape

In [110]:
df.info()

there is no null data

and data type of Gender is a object,

we need to convert it to numeric

In [111]:
df.isnull().sum()

In [112]:
df.describe()

In [113]:
# correlations
cor = df.corr()
sns.set(font_scale=1.4)
plt.figure(figsize=(9,8))
sns.heatmap(cor, annot=True, cmap='plasma')
plt.tight_layout()
plt.show()

In [114]:
# distribution plots

plt.figure(figsize=(16,12),facecolor='white')

# spending score
plt.subplot(3,3,1)
plt.title('Spending Score\n', color = '#FF000B')
sns.distplot(df['Spending Score (1-100)'], color='orange')

# age 
plt.subplot(3,3,2)
plt.title('Age\n', color = '#FF000B')
sns.distplot(df['Age'], color='#577AFF')

#Annual Income
plt.subplot(3,3,3)
plt.title('Annual Income\n', color = '#FF000B')
sns.distplot(df['Annual Income (k$)'], color = 'black')

plt.suptitle(' Distribution Plots\n', color = '#0000C1', size = 30)
plt.tight_layout()


In [115]:
# label encodel for gender 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Gender'] = le.fit_transform(df.iloc[:,0])

In [116]:
# we concerted the 'Gender' column to numeric using the 'Label Encoder'
# Male = 1, Female = 0
df.head()

In [117]:
sns.set_style('whitegrid')
plt.figure(figsize=(20, 7))
sns.countplot(x="Annual Income (k$)", data=df, palette='husl')

In [118]:
plt.rcParams['figure.figsize'] = (10, 3)
sns.countplot(df['Gender'], palette = 'Blues')
plt.title('Count Plot of Gender', fontsize = 15)
plt.show()

In [119]:
# relationship between age and spending score

plt.figure(figsize=(12,8))
sns.scatterplot(x = df['Age'], y = df['Spending Score (1-100)'])
plt.title('Age - Spending Score', size = 23, color = 'black')

**age(20-40) = area of max spendign score **

In [120]:
# relationship between annual income and spending score
plt.figure(figsize=(12,8))
sns.scatterplot(x = df['Annual Income (k$)'], y = df['Spending Score (1-100)'], palette = "red")
plt.title('Annual Income - Spending Score', size = 23, color = 'red')

**CLustering 4 variables**

**PCA**

**clustering with 4 variables, I reduce the size, because after the clustering, I may have trouble in 4D visualization while visualizing. I will do the dimension reduction with PCA, we have a dataset with 4 columns, we want to make it as many columns as we want,reduce it to 2 columns.**



In [121]:
x = df.iloc[:,0:].values
print(f'X data before PCA:\n {x[0:5]}')

In [122]:
#dff = pd.DataFrame({'A':[1,2,3],
#                   'B':[4,5,6],
#                   'C':[7,8,9],
#                   'D':[1,3,5],
#                   'E':[5,3,6],
#                   'F':[7,4,3]})

In [123]:
#print(dff)

In [124]:
#print(dff.iloc[:, :-1])

In [125]:
#X = dff.iloc[:,0:].values
#print (X)

In [126]:
# standardization before PCA

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(x)

In [127]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_2D = pca.fit_transform(X)
print(f'\nX data after PCA:\n {X_2D[0:5,:]}')

In [128]:
# finding optimum number of clusters
from sklearn.cluster import KMeans
wcss_list = []

for i in range(1,11):
    kmeans_test = KMeans(n_clusters = i, init ='k-means++', random_state=88)
    kmeans_test.fit(X_2D)
    wcss_list.append(kmeans_test.inertia_)

plt.figure(figsize=(9,6))
plt.plot(range(1, 11), wcss_list)
plt.title('The Elbow Method', color='black',fontsize='23')
plt.xlabel('Number of clusters')
plt.xticks(np.arange(1,11))
plt.ylabel('WCSS')
plt.show()

    

4 is optimum numebr of clusters

In [129]:
# KMeans
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state=88)
y_kmeans = kmeans.fit_predict(X_2D)

In [130]:
# clusters visualization
plt.figure(1 , figsize = (16 ,9))
plt.scatter(X_2D[y_kmeans == 0, 0], X_2D[y_kmeans == 0, 1], s = 80, c = 'orange', label = 'Cluster-1')
plt.scatter(X_2D[y_kmeans == 1, 0], X_2D[y_kmeans == 1, 1], s = 80, c = 'red', label = 'Cluster-2')
plt.scatter(X_2D[y_kmeans == 2, 0], X_2D[y_kmeans == 2, 1], s = 80, c = 'green', label = 'Cluster-3')
plt.scatter(X_2D[y_kmeans == 3, 0], X_2D[y_kmeans == 3, 1], s = 80, c = 'purple', label = 'Cluster-4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 375, c = 'brown', label = 'Centroids')
plt.title("Customers' Clusters")
plt.xlabel('PCA Variable-1', color='red')
plt.ylabel('PCA Variable-2', color='red')
plt.legend()
plt.show()

**Clustering Age and annual Income and Speding score**

In [131]:
# x assignment
x = df[['Age','Annual Income (k$)','Spending Score (1-100)']].values
x_df = df[['Age','Annual Income (k$)','Spending Score (1-100)']] # this line for 3d scatter plot

In [132]:
# finding optimum number of clusters
wcss_list = []

for i in range(1,11):
    kmeans_test = KMeans(n_clusters = i, init ='k-means++', random_state=88)
    kmeans_test.fit(x)
    wcss_list.append(kmeans_test.inertia_)

plt.figure(figsize=(9,6))
plt.plot(range(1, 11), wcss_list)
plt.title('The Elbow Method', color='black',fontsize='23')
plt.xlabel('Number of clusters')
plt.xticks(np.arange(1,11))
plt.ylabel('WCSS')
plt.show()

In [133]:
# KMeans
kmeans = KMeans(n_clusters = 6, init ='k-means++', random_state=88)
clusters = kmeans.fit_predict(x_df)
x_df['label'] = clusters

In [134]:
# # clusters visualization
fig = px.scatter_3d(data_frame=x_df, x='Age', y='Annual Income (k$)', z='Spending Score (1-100)',color = 'label', size = 'label')
fig.show()

**Clustering (Age and annual income)**

In [135]:
# x assignment
x = df[['Age','Annual Income (k$)']].values

In [136]:
# finding optimum number of clusters
wcss_list = []

for i in range(1,11):
    kmeans_test = KMeans(n_clusters = i, init ='k-means++', random_state=88)
    kmeans_test.fit(x)
    wcss_list.append(kmeans_test.inertia_)

plt.figure(figsize=(9,6))
plt.plot(range(1, 11), wcss_list)
plt.title('The Elbow Method', color='black',fontsize='23')
plt.xlabel('Number of clusters')
plt.xticks(np.arange(1,11))
plt.ylabel('WCSS')
plt.show()

In [137]:
# KMeans
kmeans = KMeans(n_clusters = 2, init ='k-means++', random_state=88)
y_kmeans = kmeans.fit_predict(x)

In [138]:
# clusters visualization
plt.figure(1 , figsize = (16 ,9))
plt.scatter(x[y_kmeans == 0, 0], x[y_kmeans == 0, 1], s = 80, c = '#13DB8C', label = 'Cluster-1')
plt.scatter(x[y_kmeans == 1, 0], x[y_kmeans == 1, 1], s = 80, c = '#72BAFF', label = 'Cluster-2')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 350, c = 'brown', label = 'Centroids')
plt.title("Customers' Clusters")
plt.xlabel('Age', color='red')
plt.ylabel('Annual Income (k$)', color='red')
plt.legend()
plt.show()

**Clustering by Annual Income and Spending Score**

In [139]:
# x assignment
x = df[['Annual Income (k$)','Spending Score (1-100)']].values

In [140]:
# finding optimum number of clusters
wcss_list = []

for i in range(1,11):
    kmeans_test = KMeans(n_clusters = i, init ='k-means++', random_state=88)
    kmeans_test.fit(x)
    wcss_list.append(kmeans_test.inertia_)

plt.figure(figsize=(9,6))
plt.plot(range(1, 11), wcss_list)
plt.title('The Elbow Method', color='black',fontsize='23')
plt.xlabel('Number of clusters')
plt.xticks(np.arange(1,11))
plt.ylabel('WCSS')
plt.show()

In [141]:
kmeans = KMeans(n_clusters = 5, init ='k-means++', random_state=88)
y_kmeans = kmeans.fit_predict(x)

In [142]:
# clusters visualization
plt.figure(1 , figsize = (16 ,9))
plt.scatter(x[y_kmeans == 0, 0], x[y_kmeans == 0, 1], s = 80, c = 'orange', label = 'Cluster-1')
plt.scatter(x[y_kmeans == 1, 0], x[y_kmeans == 1, 1], s = 80, c = 'red', label = 'Cluster-2')
plt.scatter(x[y_kmeans == 2, 0], x[y_kmeans == 2, 1], s = 80, c = 'purple', label = 'Cluster-3')
plt.scatter(x[y_kmeans == 3, 0], x[y_kmeans == 3, 1], s = 80, c = 'lime', label = 'Cluster-4')
plt.scatter(x[y_kmeans == 4, 0], x[y_kmeans == 4, 1], s = 80, c = 'blue', label = 'Cluster-5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 375, c = 'brown', label = 'Centroids')
plt.title("Customers' Clusters")
plt.xlabel('Annual Income (k$)', color='red')
plt.ylabel('Spending Score', color='red')
plt.legend()
plt.show()

**Clustering by Age and Spending Score

In [143]:
# x assignment
x = df[['Age','Spending Score (1-100)']].values

In [144]:
# finding optimum number of clusters
wcss_list = []

for i in range(1,11):
    kmeans_test = KMeans(n_clusters = i, init ='k-means++', random_state=88)
    kmeans_test.fit(x)
    wcss_list.append(kmeans_test.inertia_)

plt.figure(figsize=(9,6))
plt.plot(range(1, 11), wcss_list)
plt.title('The Elbow Method', color='black',fontsize='23')
plt.xlabel('Number of clusters')
plt.xticks(np.arange(1,11))
plt.ylabel('WCSS')
plt.show()

In [145]:
# KMeans
kmeans = KMeans(n_clusters = 4, init ='k-means++', random_state=88)
y_kmeans = kmeans.fit_predict(x)

In [146]:
# clusters visualization
plt.figure(1 , figsize = (16 ,9))
plt.scatter(x[y_kmeans == 0, 0], x[y_kmeans == 0, 1], s = 80, c = '#00FF00', label = 'Cluster-1')
plt.scatter(x[y_kmeans == 1, 0], x[y_kmeans == 1, 1], s = 80, c = '#00FFFF', label = 'Cluster-2')
plt.scatter(x[y_kmeans == 2, 0], x[y_kmeans == 2, 1], s = 80, c = '#FF00FF', label = 'Cluster-3')
plt.scatter(x[y_kmeans == 3, 0], x[y_kmeans == 3, 1], s = 80, c = '#FF4500', label = 'Cluster-4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 375, c = 'brown', label = 'Centroids')
plt.title("Customers' Clusters")
plt.xlabel('Age', color='red')
plt.ylabel('Spending Score', color='red')
plt.legend()
plt.show()